In [6]:
import collections
import datetime
import fluidsynth
import glob
import numpy as np
import pathlib
import pandas as pd
import pretty_midi
import seaborn as sns
import tensorflow as tf

from IPython import display
from matplotlib import pyplot as plt
from typing import Optional

seed = 42
tf.random.set_seed(seed)
np.random.seed(seed)

# Sampling rate for audio playback
_SAMPLING_RATE = 16000

def display_audio(pm: pretty_midi.PrettyMIDI, seconds=30):
      waveform = pm.fluidsynth(fs=_SAMPLING_RATE)
      # Take a sample of the generated waveform to mitigate kernel resets
      waveform_short = waveform[:seconds*_SAMPLING_RATE]
      return display.Audio(waveform_short, rate=_SAMPLING_RATE, autoplay=True)

data_dir = pathlib.Path('data/maestro-v2.0.0')

filenames = glob.glob(str(data_dir/'**/*.mid*'))
print('Number of files:', len(filenames))

sample_file = filenames[1]
print(sample_file)

pm = pretty_midi.PrettyMIDI(sample_file) 
# waveform = pm.fluidsynth(fs=_SAMPLING_RATE)
# # Take a sample of the generated waveform to mitigate kernel resets
# waveform_short = waveform[:30*_SAMPLING_RATE]
# t = display.Audio(waveform_short, rate=_SAMPLING_RATE, autoplay=True)
# t

aud  = display_audio(pm)
aud

# print('Number of instruments:', len(pm.instruments))
# instrument = pm.instruments[0]
# instrument_name = pretty_midi.program_to_instrument_name(instrument.program)
# print('Instrument name:', instrument_name)

# for i, note in enumerate(instrument.notes[:10]):
#   note_name = pretty_midi.note_number_to_name(note.pitch)
#   duration = note.end - note.start
#   print(f'{i}: pitch={note.pitch}, note_name={note_name},'
#         f' duration={duration:.4f}')

Number of files: 1282
data/maestro-v2.0.0/2013/ORIG-MIDI_03_7_6_13_Group__MID--AUDIO_09_R1_2013_wav--2.midi
